[this doc on github](https://github.com/dotnet/interactive/tree/main/samples/notebooks/csharp/Samples)

# .NET Interactive report 

project report for [.NET Interactive repo](https://github.com/dotnet/interactive)

## Setup
Importing pacakges and setting up connection

In [ ]:
#r "nuget: NodaTime, 3.1.0"
#r "nuget: Octokit, 0.51.0"
#r "nuget: Plotly.NET, 2.0.0"
#r "nuget: Plotly.NET.Interactive, 2.0.0"

using Microsoft.FSharp.Core;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Octokit;
using NodaTime;
using NodaTime.Extensions;
using Plotly.NET;
using Plotly.NET.LayoutObjects;

In [ ]:
var organization = "dotnet";
var repositoryName = "interactive";
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

[Generate a user token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line) to get rid of public [api](https://github.com/octokit/octokit.net/blob/master/docs/getting-started.md) throttling policies for anonymous users 

In [ ]:
var tokenAuth = new Credentials("your token");
gitHubClient.Credentials = tokenAuth;

In [ ]:
var today = SystemClock.Instance.InUtc().GetCurrentDate();
var startOfTheMonth = today.With(DateAdjusters.StartOfMonth);
var startOfPreviousMonth = today.With(DateAdjusters.StartOfMonth) - Period.FromMonths(1);
var startOfTheYear = new LocalDate(today.Year, 1, 1).AtMidnight();

var currentYearIssuesRequest = new RepositoryIssueRequest {
     State = ItemStateFilter.All,
     Since = startOfTheYear.ToDateTimeUnspecified()
};

var pullRequestRequest = new PullRequestRequest {
    State = ItemStateFilter.All
};

Perform github queries

In [ ]:
#!time
var branches = await gitHubClient.Repository.Branch.GetAll(organization, repositoryName);
var pullRequests = await gitHubClient.Repository.PullRequest.GetAllForRepository(organization, repositoryName, pullRequestRequest);
var forks = await gitHubClient.Repository.Forks.GetAll(organization, repositoryName);
var currentYearIssues = await gitHubClient.Issue.GetAllForRepository(organization, repositoryName, currentYearIssuesRequest);

Branch data

Pull request data

In [ ]:
var pullRequestCreatedThisMonth = pullRequests.Where(pr => pr.CreatedAt > startOfTheMonth.ToDateTimeUnspecified());
var pullRequestClosedThisMonth =pullRequests.Where(pr => (pr.MergedAt != null && pr.MergedAt > startOfTheMonth.ToDateTimeUnspecified()));
var contributorsCount = pullRequestClosedThisMonth.GroupBy(pr => pr.User.Login);

var pullRequestLifespan = pullRequests.GroupBy(pr =>
            {
                var lifeSpan = (pr.ClosedAt ?? today.ToDateTimeUnspecified()) - pr.CreatedAt;
                return Math.Max(0, Math.Ceiling(lifeSpan.TotalDays));
            })
            .Where(g => g.Key > 0)
            .OrderBy(g => g.Key)
            .ToDictionary(g => g.Key, g => g.Count());

Fork data

In [ ]:
var forkCreatedThisMonth = forks.Where(fork => fork.CreatedAt >= startOfTheMonth.ToDateTimeUnspecified());
var forkCreatedPreviousMonth = forks.Where(fork => (fork.CreatedAt >= startOfPreviousMonth.ToDateTimeUnspecified()) && (fork.CreatedAt < startOfTheMonth.ToDateTimeUnspecified()));
var forkCreatedByMonth = forks.GroupBy(fork => new DateTime(fork.CreatedAt.Year, fork.CreatedAt.Month, 1));
var forkUpdateByMonth = forks.GroupBy(f => new DateTime(f.UpdatedAt.Year, f.UpdatedAt.Month,  1) ).Select(g => new {Date = g.Key, Count = g.Count()}).OrderBy(g => g.Date).ToArray();
var total = 0;
var forkCountByMonth = forkCreatedByMonth.OrderBy(g => g.Key).Select(g => new {Date = g.Key, Count = total += g.Count()}).ToArray();

Issues data

In [ ]:
bool IsBug(Issue issue){
    return issue.Labels.FirstOrDefault(l => l.Name == "bug")!= null;
}

bool TargetsArea(Issue issue){
    return issue.Labels.FirstOrDefault(l => l.Name.StartsWith("Area-"))!= null;
}

string GetArea(Issue issue){
    return issue.Labels.FirstOrDefault(l => l.Name.StartsWith("Area-"))?.Name;
}

var openIssues = currentYearIssues.Where(IsBug).Where(issue => issue.State == "open");
var closedIssues = currentYearIssues.Where(IsBug).Where(issue => issue.State == "closed");
var oldestIssues = openIssues.OrderBy(issue => today.ToDateTimeUnspecified() - issue.CreatedAt).Take(20);
var createdCurrentMonth = currentYearIssues.Where(IsBug).Where(issue => issue.CreatedAt >= startOfTheMonth.ToDateTimeUnspecified());
var createdPreviousMonth = currentYearIssues.Where(IsBug).Where(issue => (issue.CreatedAt >= startOfPreviousMonth.ToDateTimeUnspecified()) && (issue.CreatedAt < startOfTheMonth.ToDateTimeUnspecified()));
var openFromPreviousMonth = openIssues.Where(issue => (issue.CreatedAt > startOfPreviousMonth.ToDateTimeUnspecified()) && (issue.CreatedAt < startOfTheMonth.ToDateTimeUnspecified()));
var createdByMonth = currentYearIssues.Where(IsBug).GroupBy(issue => new DateTime(issue.CreatedAt.Year, issue.CreatedAt.Month, 1)).OrderBy(g=>g.Key).ToDictionary(g => g.Key, g => g.Count());
var closedByMonth = closedIssues.GroupBy(issue => new DateTime((int) issue.ClosedAt?.Year, (int) issue.ClosedAt?.Month, 1)).OrderBy(g=>g.Key).ToDictionary(g => g.Key, g => g.Count());
var openIssueAge = openIssues.GroupBy(issue => new DateTime(issue.CreatedAt.Year, issue.CreatedAt.Month, issue.CreatedAt.Day)).ToDictionary(g => g.Key, g => g.Max(issue =>Math.Max(0, Math.Ceiling( (today.ToDateTimeUnspecified() - issue.CreatedAt).TotalDays))));
var openByMonth = new Dictionary<DateTime, int>();
var minDate = createdByMonth.Min(g => g.Key);
var maxCreatedAtDate = createdByMonth.Max(g => g.Key);
var maxClosedAtDate = closedByMonth.Max(g => g.Key);
var maxDate = maxCreatedAtDate > maxClosedAtDate ?maxCreatedAtDate : maxClosedAtDate;
var cursor = minDate;
var runningTotal = 0;
var issuesCreatedThisMonthByArea = currentYearIssues.Where(issue => issue.CreatedAt >= startOfTheMonth.ToDateTimeUnspecified()).Where(issue => IsBug(issue) && TargetsArea(issue)).GroupBy(issue => GetArea(issue)).ToDictionary(g => g.Key, g => g.Count());
var openIssueByArea = currentYearIssues.Where(issue => issue.State == "open").Where(issue => IsBug(issue) && TargetsArea(issue)).GroupBy(issue => GetArea(issue)).ToDictionary(g => g.Key, g => g.Count());

while (cursor <= maxDate )
{
    createdByMonth.TryGetValue(cursor, out var openCount);
    closedByMonth.TryGetValue(cursor, out var closedCount);
    runningTotal += (openCount - closedCount);
    openByMonth[cursor] = runningTotal;
    cursor = cursor.AddMonths(1);
}

var issueLifespan = currentYearIssues.Where(IsBug).GroupBy(issue =>
            {
                var lifeSpan = (issue.ClosedAt ?? today.ToDateTimeUnspecified()) - issue.CreatedAt;
                return Math.Max(0, Math.Round(Math.Ceiling(lifeSpan.TotalDays),0));
            })
            .Where(g => g.Key > 0)
            .OrderBy(g => g.Key)
            .ToDictionary(g => g.Key, g => g.Count());

display(new { 
    less_then_one_sprint = issueLifespan.Where(i=> i.Key < 21).Select(i => i.Value).Sum(),
    less_then_two_sprint = issueLifespan.Where(i=> i.Key >= 21 && i.Key < 42).Select(i => i.Value).Sum(),
    more_then_two_sprint = issueLifespan.Where(i=> i.Key >= 42).Select(i => i.Value).Sum()   
    });

# Activity dashboard

In [ ]:
GenericChart.GenericChart ScatterglDict<T1, T2>(Dictionary<T1, T2> dict, string name)
{
    var trace = new Trace("scatter");
    trace.SetValue("x", dict.Select(pair => pair.Key));
    trace.SetValue("y",  dict.Select(pair => pair.Value));
    trace.SetValue("name", name);
    return GenericChart.ofTraceObject(true, trace);
}

var issueChart = GenericChart.combine( new []{
    ScatterglDict(createdByMonth, "Created"), 
    ScatterglDict(openByMonth, "Open"), 
    ScatterglDict(closedByMonth, "Closed")}
);

issueChart.WithTitle("Bugs by month");
issueChart.Display();

In [ ]:
// Helper function for some of the below charts.

GenericChart.GenericChart BarIEnumerableKeyValuePair<T1,T2>(IEnumerable<KeyValuePair<T1,T2>> seq, string name, string color)
{
    var trace = new Trace("bar");
    trace.SetValue("x", seq.OrderBy(issue => issue.Key).Select(issue => issue.Value));
    trace.SetValue("y",  seq.OrderBy(issue => issue.Key).Select(issue => issue.Key));
    trace.SetValue("name", name);
    return GenericChart.ofTraceObject(true, trace);

    // return new Bar
    // {
    //     name = name,
    //     y = seq.OrderBy(issue => issue.Key).Select(issue => issue.Value),
    //     x = seq.OrderBy(issue => issue.Key).Select(issue => issue.Key),
    //     marker = new Marker{ color = color }  
    // };    
}

In [ ]:
var issueLifespanChart =  GenericChart.combine(new[] {
    BarIEnumerableKeyValuePair(issueLifespan.Where(issue => issue.Key < 7), "One week old", "green"), 
    BarIEnumerableKeyValuePair(issueLifespan.Where(issue => issue.Key >= 7 && issue.Key < 21), "One Sprint old", "yellow"), 
    BarIEnumerableKeyValuePair(issueLifespan.Where(issue => issue.Key >= 21), "More then a Sprint", "red")
});


issueLifespanChart.WithTitle( "Bugs by life span");
issueLifespanChart.WithXAxisStyle(Title.init("Bugs by life span"));

issueLifespanChart.Display();

In [ ]:
var openIssuesAgeChart = GenericChart.combine(new[] {
    BarIEnumerableKeyValuePair(openIssueAge.Where(issue => issue.Value < 7), "Closed in a week", "green"), 
    BarIEnumerableKeyValuePair(openIssueAge.Where(issue => issue.Value >= 7 && issue.Value < 21), "Closed within a sprint", "yellow"), 
    BarIEnumerableKeyValuePair(openIssueAge.Where(issue => issue.Value >= 21), "Long standing", "red")
});

openIssuesAgeChart.WithTitle( "Open bugs age");
openIssuesAgeChart.WithXAxisStyle(Title.init("Number of days a bug stays open"));

openIssuesAgeChart.Display();


In [ ]:
var createdThisMonthAreaSeries = new Trace("pie");
createdThisMonthAreaSeries.SetValue("values", issuesCreatedThisMonthByArea.Select(issue => issue.Value));
createdThisMonthAreaSeries.SetValue("labels",  issuesCreatedThisMonthByArea.Select(e => e.Key));


var createdArea = Chart.Combine(new[] {GenericChart.ofTraceObject(true, createdThisMonthAreaSeries)});
createdArea.WithTitle( "Bugs created this month by Area");

createdArea.Display();

In [ ]:
var openAreaSeries = new Trace("pie");
openAreaSeries.SetValue("values", openIssueByArea.Select(e => e.Value));
openAreaSeries.SetValue("labels",  openIssueByArea.Select(e => e.Key));


var openArea = Chart.Combine(new[] {GenericChart.ofTraceObject(true, openAreaSeries)});
openArea.WithTitle( "Open bugs by Area");

openArea.Display();

In [ ]:
var prLifespanChart = Chart.Combine(new[] {
    BarIEnumerableKeyValuePair(pullRequestLifespan.Where(issue => issue.Key < 7), "One week", "green"), 
    BarIEnumerableKeyValuePair(pullRequestLifespan.Where(issue => issue.Key >= 7 && issue.Key < 21), "One Sprint", "yellow"), 
    BarIEnumerableKeyValuePair(pullRequestLifespan.Where(issue => issue.Key >= 21), "More than a Sprint", "red")
});

prLifespanChart.WithTitle( "Pull Request by life span");
prLifespanChart.WithXAxisStyle(Title.init("Number of days a PR stays open"));
prLifespanChart.WithYAxisStyle(Title.init("Number of PR"));
prLifespanChart.Display();

In [ ]:
var forkCreationSeries = new Trace("scatter");
forkCreationSeries.SetValue("x", forkCreatedByMonth.Select(g => g.Key ).ToArray());
forkCreationSeries.SetValue("y", forkCreatedByMonth.Select(g => g.Count() ).ToArray());
forkCreationSeries.SetValue("name", "created by month");

var forkTotalSeries = new Trace("scatter");
forkTotalSeries.SetValue("x", forkCountByMonth.Select(g => g.Date ).ToArray());
forkTotalSeries.SetValue("y", forkCountByMonth.Select(g => g.Count ).ToArray());
forkTotalSeries.SetValue("name", "running total");


var forkUpdateSeries = new Trace("scatter");
forkUpdateSeries.SetValue("x", forkUpdateByMonth.Select(g => g.Date ).ToArray());
forkUpdateSeries.SetValue("y", forkUpdateByMonth.Select(g => g.Count ).ToArray());
forkUpdateSeries.SetValue("name", "last update by month");




var chart = Chart.Combine(new[] {GenericChart.ofTraceObject(true,forkCreationSeries),GenericChart.ofTraceObject(true,forkTotalSeries),GenericChart.ofTraceObject(true,forkUpdateSeries)});
chart.WithTitle("Fork activity");
chart.Display();